In [4]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import polars as pl

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

CODIGO_ROBERTO = 109101
CODIGO_EXEQUIEL = 113130
CODIGO_CALVO = 112102
CODIGO_INT = 112103

CODIGOS_PEDIATRICOS = [CODIGO_ROBERTO, CODIGO_EXEQUIEL, CODIGO_CALVO]

In [25]:
df_pediatricos = (
    pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
    .filter(
        (pl.col("COD_HOSPITAL").is_in(CODIGOS_PEDIATRICOS))
        & (pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN")
    )
    .group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL"]))
    .agg(
        pl.col("DIAGNOSTICO1").count().alias("n_egresos"),
        pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
        pl.col("IR_29301_PESO").mean().alias("promedio_peso_grd"),
    )
    .sort(by=["COD_HOSPITAL", "ANIO_EGRESO", "n_egresos"], descending=False)
).collect().to_pandas()

In [51]:
comparacion_pesos_pediatricos = pd.pivot_table(
    df_pediatricos,
    values=["n_egresos", "dias_estada_totales", "promedio_peso_grd"],
    index=["COD_HOSPITAL"],
    columns=["ANIO_EGRESO"],
    aggfunc="sum"
)

In [52]:
comparacion_pesos_pediatricos

dias_estada_totales                             n_egresos        \
ANIO_EGRESO                 2019   2020   2021   2022   2023      2019  2020   
COD_HOSPITAL                                                                   
109101                     51598  35201  41939  45218  47892      9200  4960   
112102                     50801  42709  48518  47070  48336      8424  5308   
113130                     46371  31534  39876  39975  40706      7329  4866   

                               promedio_peso_grd                      \
ANIO_EGRESO   2021  2022  2023              2019      2020      2021   
COD_HOSPITAL                                                           
109101        5640  6890  7723          1.109254  1.329573  1.271459   
112102        6771  7045  7519          1.418824  1.627579  1.617592   
113130        6302  7074  7401          1.019888  1.147688  1.121131   

                                  
ANIO_EGRESO       2022      2023  
COD_HOSPITAL                      
109101        1.258852  1.195931  
112102        1.517342  1.389222  
113130        1.117089  1.085647

In [37]:
df = (
    (
        pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
        .filter((pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
        .with_columns(pl.col("IR_29301_PESO").cast(pl.Float64))
        .group_by(pl.col(["ANIO_EGRESO"]))
        .agg(
            pl.col("DIAGNOSTICO1").count().alias("n_egresos"),
            pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
            pl.col("IR_29301_PESO").fill_null(0).mean().alias("promedio_peso_grd"),
        )
        .sort(by=["ANIO_EGRESO", "n_egresos"], descending=False)
    )
    .collect()
    .to_pandas()
)

In [39]:
df = pd.read_csv("../data/processed/df_procesada.csv", sep=";")

C:\Users\jarojas\AppData\Local\Temp\ipykernel_14252\1252128792.py:1: DtypeWarning: Columns (1,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,101,105,113,123,124,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/df_procesada.csv", sep=";")


In [43]:
comparacion_pesos_pais = (
    df.query("TIPO_ACTIVIDAD == 'HOSPITALIZACIÓN'")
    .groupby(["ANIO_EGRESO"])
    .agg(
        n_egresos=("DIAGNOSTICO1", "count"),
        dias_estada_totales=("ESTANCIA", "sum"),
        promedio_peso_grd=("IR_29301_PESO", "mean"),
    )
)

In [45]:
comparacion_pesos_pais["COD_HOSPITAL"] = "Pais"

In [47]:
comparacion_pesos_pais = comparacion_pesos_pais.reset_index()

In [60]:
comparacion_pesos_pais["ANIO_EGRESO"] = comparacion_pesos_pais["ANIO_EGRESO"].astype(int)

In [61]:
fila_pesos_pais = pd.pivot_table(
    comparacion_pesos_pais,
    index="COD_HOSPITAL",
    values=["n_egresos", "dias_estada_totales", "promedio_peso_grd"],
    columns="ANIO_EGRESO",
    aggfunc="sum",
)

In [64]:
comparacion_pesos_pais

,ANIO_EGRESO,n_egresos,dias_estada_totales,promedio_peso_grd,COD_HOSPITAL
0,2019,886166,5994123.0,0.927393,Pais
1,2020,702098,5178989.0,1.058420,Pais
2,2021,713946,5636831.0,1.184195,Pais
3,2022,777974,5795982.0,1.058281,Pais
4,2023,846010,6054140.0,1.054084,Pais


In [66]:
pd.concat([comparacion_pesos_pediatricos, fila_pesos_pais]).to_excel("comparacion_complejidad_GRD.xlsx")